In [1]:
import pandas as pd
import numpy as np
import warnings
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
warnings.filterwarnings("ignore")
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
# Modèles
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors

# Ce dont on a besoin
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score, GridSearchCV

In [4]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression

In [5]:
chemin_bd = r"./bd_ignore/"

In [20]:
df_merge_recom22 = pd.read_csv(chemin_bd+"resultat2/df_merge_recom21.csv")

In [ ]:
df_merge_recom22 =df_merge_recom22.drop(columns=['startYear','homepage','overview','poster_path'])

In [ ]:
df_merge_recom22 = df_merge_recom22.fillna('None')

In [22]:
merge_filmR = df_merge_recom22[df_merge_recom22['titleType'] == 'movie']
merge_serieR = df_merge_recom22[df_merge_recom22['titleType'] == 'tvSeries']

In [ ]:
agg_df = df_merge_recom22.groupby(['original_title'])['primaryName'].agg(list).reset_index()
agg_df

In [ ]:
# Agrégation de primaryName par original_title
#agg_df = df_merge_recom22.groupby(['original_title'])['primaryName'].agg(list).reset_index()

# Merge pour conserver toutes les colonnes originales
df_merge_recom22 = df_merge_recom22.drop_duplicates(subset=['original_title']).merge(agg_df, on='original_title', how='left')

# Transformation de genres_y en listes
df_merge_recom22['genres_liste'] = df_merge_recom22['genres_y'].apply(lambda x: x.split(","))

In [ ]:
agg_df['primaryName']=agg_df['primaryName'].apply(lambda x: x[0 : 2])
agg_df

In [23]:
caracteristiques = ['titleType', 'startYear', 'genres_y', 'primaryName']

In [ ]:
X = merge_filmR[caracteristiques]

In [9]:
merge_filmR['homepage'] = merge_filmR['homepage'].fillna('None')
merge_filmR['poster_path'] = merge_filmR['poster_path'].fillna('None')
merge_filmR['overview'] = merge_filmR['overview'].fillna('None')
merge_filmR = merge_filmR.fillna('None')

In [11]:
df_merge_recom22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1312795 entries, 0 to 1312794
Data columns (total 9 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   homepage        1312795 non-null  object 
 1   imdb_id         1312795 non-null  object 
 2   original_title  1312795 non-null  object 
 3   overview        1312795 non-null  object 
 4   poster_path     1312795 non-null  object 
 5   titleType       1312795 non-null  object 
 6   startYear       1312795 non-null  float64
 7   genres_y        1312795 non-null  object 
 8   primaryName     1312795 non-null  object 
dtypes: float64(1), object(8)
memory usage: 90.1+ MB


In [12]:
df_merge_recom22.columns

Index(['homepage', 'imdb_id', 'original_title', 'overview', 'poster_path',
       'titleType', 'startYear', 'genres_y', 'primaryName'],
      dtype='object')

In [13]:
agg_df = df_merge_recom22.groupby(['original_title'])['primaryName'].agg(list).reset_index()
agg_df

,original_title,primaryName
0,"""BLESSED""","[Kevin Bradley, Jason Dill, Sage Elsesser, Mar..."
1,"""Gumdrop"", a short horror","[Tom Remick, Stan Wankowski, Luke Remick, Dani..."
2,"""それ"" がいる森","[Masaki Aiba, Noriko Eguchi, Fumiyo Kohinata, ..."
3,"""隠れビッチ"" やってました。","[Yui Sakuma, Nijirô Murakami, Suzuka Ohgo, Yût..."
4,#1,"[Austin Adelman, Alden Moody, Jennifer Lundin,..."
...,...,...
144204,ＳＭ集団（ロウ）責め,"[Rio Serizawa, Yutaka Ikejima, Momori Asano, K..."
144205,Ｗの女 幻妖の甘い罠,"[Meguri, Rin Aoki, Shôko Kudô, Takashi Naha, S..."
144206,ＸＸ 美しき獲物,"[Kei Marimura, Makiko Watanabe, Ren Ôsugi, Ats..."
144207,ｄビデオスペシャル 仮面ライダー4号,"[Ryoma Takeuchi, Ryoma Takeuchi, Rio Uchida, Y..."


In [14]:
agg_df['primaryName']=agg_df['primaryName'].apply(lambda x: x[0 : 2])
agg_df

,original_title,primaryName
0,"""BLESSED""","[Kevin Bradley, Jason Dill]"
1,"""Gumdrop"", a short horror","[Tom Remick, Stan Wankowski]"
2,"""それ"" がいる森","[Masaki Aiba, Noriko Eguchi]"
3,"""隠れビッチ"" やってました。","[Yui Sakuma, Nijirô Murakami]"
4,#1,"[Austin Adelman, Alden Moody]"
...,...,...
144204,ＳＭ集団（ロウ）責め,"[Rio Serizawa, Yutaka Ikejima]"
144205,Ｗの女 幻妖の甘い罠,"[Meguri, Rin Aoki]"
144206,ＸＸ 美しき獲物,"[Kei Marimura, Makiko Watanabe]"
144207,ｄビデオスペシャル 仮面ライダー4号,"[Ryoma Takeuchi, Ryoma Takeuchi]"


In [15]:
# Merge pour conserver toutes les colonnes originales
df_merge_recom22 = df_merge_recom22.drop_duplicates(subset=['original_title']).merge(agg_df, on='original_title', how='left')

# Transformation de genres_y en listes
df_merge_recom22['genres_liste'] = df_merge_recom22['genres_y'].apply(lambda x: x.split(","))

In [15]:
df_merge_recom22['genres_liste']

0                            [Comedy]
1                            [Comedy]
2                            [Comedy]
3                            [Comedy]
4                            [Comedy]
                      ...            
1312790    [Action, Adventure, Drama]
1312791    [Action, Adventure, Drama]
1312792    [Action, Adventure, Drama]
1312793    [Action, Adventure, Drama]
1312794    [Action, Adventure, Drama]
Name: genres_liste, Length: 1312795, dtype: object

In [134]:
# def transfo_liste(x):
#   if isinstance(x, str):
#     return ast.literal_eval(x)
#   else:
#     return x

# -> fonction de jo 

In [135]:
# transformer la colonne genre en liste
#df_merge_recom22['genres_liste'] = df_merge_recom22['genres_liste'].apply(transfo_liste)
#df_merge_recom22.sample(5)
# je force la colonne genre_liste à n'être que des listes

In [16]:
tous_les_genres = set()
tous_les_acteurs = set()

In [17]:
for genres in df_merge_recom22['genres_liste']:
  tous_les_genres.update(genres)

tous_les_genres
#ensemble qui contient tous mes genres

{'Action',
 'Adult',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Game-Show',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'News',
 'Reality-TV',
 'Romance',
 'Sci-Fi',
 'Short',
 'Sport',
 'Talk-Show',
 'Thriller',
 'War',
 'Western',
 '\\N'}

In [18]:
# Mise à jour de l'ensemble des acteurs
for acteurs in df_merge_recom22['primaryName_y']:
    tous_les_acteurs.update(acteurs)

In [19]:
for genre in tous_les_genres:
  df_merge_recom22[f'genre_{genre}'] = df_merge_recom22['genres_liste'].apply(lambda x: genre in x)

df_merge_recom22.sample(3)

,homepage,imdb_id,original_title,overview,poster_path,titleType,startYear,genres_y,primaryName_x,primaryName_y,...,genre_Game-Show,genre_Animation,genre_\N,genre_Sci-Fi,genre_Comedy,genre_Adult,genre_Fantasy,genre_Family,genre_Action,genre_Adventure
86530,https://opprime.tv/video/668,tt15155264,Love Mementos,Young professional lady realizes her fiance i...,/hqDE8rikgiCZ8prHxEda8216YyD.jpg,short,2020.0,"Drama,Romance,Short",Emery Nguyen,[Emery Nguyen],...,False,False,False,False,False,False,False,False,False,False
13806,None,tt7629058,Водата,The water is a metaphorical view of life in th...,/u8lhWwsfKzypTHpaM6Iu48WPaJs.jpg,short,2018.0,"Documentary,Short",Cristo Fernández,[Cristo Fernández],...,False,False,False,False,False,False,False,False,False,False
97595,None,tt0420683,I Do (but I Don't),"Denise Richards plays Lauren, a divorced weddi...",/jHcaovdkQB390yCh0aYzvjBjMJP.jpg,tvMovie,2004.0,"Comedy,Drama,Romance",Denise Richards,"[Denise Richards, Dean Cain]",...,False,False,False,False,True,False,False,False,False,False


In [ ]:
# Mise à jour de l'ensemble des acteurs
for acteurs in df_merge_recom22['primaryName_y']: #ne marche pas ///////////////////////////////////////////////////
    tous_les_acteurs.update(acteurs)

In [19]:
#df avec tous les genres en booleens

df_merge_recom22.head(5)

,homepage,imdb_id,original_title,overview,poster_path,titleType,startYear,genres_y,primaryName,genres_liste,...,genre_History,genre_\N,genre_Western,genre_Crime,genre_Thriller,genre_Horror,genre_Drama,genre_Adventure,genre_Animation,genre_Romance
0,https://www.miramax.com/movie/four-rooms/,tt0113101,Four Rooms,It's Ted the Bellhop's first night on the job....,/75aHn1NOYXh4M7L5shoeQ6NGykP.jpg,movie,1995.0,Comedy,Tim Roth,[Comedy],...,False,False,False,False,False,False,False,False,False,False
1,https://www.miramax.com/movie/four-rooms/,tt0113101,Four Rooms,It's Ted the Bellhop's first night on the job....,/75aHn1NOYXh4M7L5shoeQ6NGykP.jpg,movie,1995.0,Comedy,Antonio Banderas,[Comedy],...,False,False,False,False,False,False,False,False,False,False
2,https://www.miramax.com/movie/four-rooms/,tt0113101,Four Rooms,It's Ted the Bellhop's first night on the job....,/75aHn1NOYXh4M7L5shoeQ6NGykP.jpg,movie,1995.0,Comedy,Sammi Davis,[Comedy],...,False,False,False,False,False,False,False,False,False,False
3,https://www.miramax.com/movie/four-rooms/,tt0113101,Four Rooms,It's Ted the Bellhop's first night on the job....,/75aHn1NOYXh4M7L5shoeQ6NGykP.jpg,movie,1995.0,Comedy,Amanda De Cadenet,[Comedy],...,False,False,False,False,False,False,False,False,False,False
4,https://www.miramax.com/movie/four-rooms/,tt0113101,Four Rooms,It's Ted the Bellhop's first night on the job....,/75aHn1NOYXh4M7L5shoeQ6NGykP.jpg,movie,1995.0,Comedy,Valeria Golino,[Comedy],...,False,False,False,False,False,False,False,False,False,False


In [20]:
df_merge_recom22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1312795 entries, 0 to 1312794
Data columns (total 38 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   homepage           1312795 non-null  object 
 1   imdb_id            1312795 non-null  object 
 2   original_title     1312795 non-null  object 
 3   overview           1312795 non-null  object 
 4   poster_path        1312795 non-null  object 
 5   titleType          1312795 non-null  object 
 6   startYear          1312795 non-null  float64
 7   genres_y           1312795 non-null  object 
 8   primaryName        1312795 non-null  object 
 9   genres_liste       1312795 non-null  object 
 10  genre_Game-Show    1312795 non-null  bool   
 11  genre_Sci-Fi       1312795 non-null  bool   
 12  genre_Adult        1312795 non-null  bool   
 13  genre_Reality-TV   1312795 non-null  bool   
 14  genre_Mystery      1312795 non-null  bool   
 15  genre_War          1312795 non-n

In [ ]:
# # commence le test pour regrouper les acteurs dans une même cellule et donc n'avoir plus qu'une ligne par film

# df_merge_recom22_actors = df_merge_recom22.copy()

# # groupby sur notre df_recom22 sur la colonne imdb_id tout en appliquant une fonction lambda pour regrouper les acteurs en les séparant d'une virgule (fonction lambda s'applique par film grâce au groupby)

# df_merge_recom22_actors = pd.DataFrame(df_merge_recom22_actors.groupby('imdb_id', sort = False, as_index = False)['primaryName'].apply(lambda x: "%s" % ', '.join(x)))

# # on merge mtn ce petit df d'acteurs sur notre df_recom22 via l'imdb_id

# df_merge_recom22_final = df_merge_recom22.merge(df_merge_recom22_actors, how='inner', on='imdb_id', sort=False, copy=False)


# # je supprime ici la colonne du nom des acteurs du df_recom22 puisqu'on a mtn la nouvelle avec les acterus regroupés
# df_merge_recom22_final = df_merge_recom22_final.drop('primaryName_x', axis=1)


# # n'ayant plus la colonne d'acteurs du df de base, toutes les lignes d'un même film sont devenues similaires en tout point
# # nous pouvons alors supprimer les lignes dupliquées

# df_merge_recom22_final = df_merge_recom22_final.drop_duplicates(subset='imdb_id').reset_index()

# # df avec une ligne par film et la colonne acteur regroupant tous les acteurs d'un même film

# df_merge_recom22_final

# # df_merge_recom22_final['acteurs_liste'] = df_merge_recom22_final['primaryName_y'].apply(lambda x: x.split(","))
# # tous_les_acteurs = set()

# # for acteurs in df_merge_recom22_final['acteurs_liste']:
# #   tous_les_acteurs.update(acteurs)

# # tous_les_acteurs

In [ ]:
# à ne pas lancer ça tourne pendant des heures
# for acteur in tous_les_acteurs:
#   df_merge_recom22_final[f'act_{acteur}'] = df_merge_recom22_final['acteurs_liste'].apply(lambda x: acteur in x)

# df_merge_recom22_final.sample(3)

In [23]:
merge_filmR = df_merge_recom22[df_merge_recom22['titleType'] == 'movie']
merge_serieR = df_merge_recom22[df_merge_recom22['titleType'] == 'tvSeries']

In [24]:
#caracteristiques = ['titleType', 'startYear', 'genres_y', 'primaryName']
caracteristiques = ['titleType', 'startYear']

In [25]:
X = merge_filmR[caracteristiques]

In [26]:
X_genre = merge_filmR.select_dtypes('bool')

In [27]:
def encodage_X(X, type='standard'):
  index = X.index
  X_num = X.select_dtypes('number')
  X_cat = X.select_dtypes(['object', 'category', 'string'])
  # Comme ça les dates, on n'y touche pas

  if type == 'standard':
    from sklearn.preprocessing import StandardScaler
    SN = StandardScaler()
    X_num_SN = pd.DataFrame(SN.fit_transform(X_num), columns=X_num.columns, index=index)

  else:
    from sklearn.preprocessing import MinMaxScaler
    SN = MinMaxScaler()
    X_num_SN = pd.DataFrame(SN.fit_transform(X_num), columns=X_num.columns, index=index)

  X_cat_dummies = pd.get_dummies(X_cat)
  X_encoded = pd.concat([X_num_SN, X_cat_dummies], axis=1)
  X_encoded = pd.concat([X_encoded, X_genre], axis=1)

  return X_encoded, SN

In [28]:
X_encoded, SN = encodage_X(X, type='normalisation')

In [29]:
def film_similaire(nom_film):

  # Vérifier si le film existe dans le dataset
  if nom_film not in merge_filmR['imdb_id'].values:
      return f"Le film numero {nom_film} n'est pas dans le dataset."

  # Récupérer les caractéristiques du Pokémon
  filmC = merge_filmR[merge_filmR['imdb_id'] == nom_film]

  # Vérifier si le Pokémon est légendaire
  if not merge_filmR['imdb_id'].iloc[0]:
      return f"{nom_film} n'est pas un film dans le dataset."

  # Je recopie ce qu'on a fait avant:
  caract_film = merge_filmR[merge_filmR['imdb_id'] == nom_film][caracteristiques]

  caract_film_encoded = encodage_predict(caract_film)

  distances, indices = model.kneighbors(caract_film_encoded)

  return merge_filmR.iloc[indices[0]].reset_index(drop=True)

: 

In [ ]:
index = X.index
X_num = X.select_dtypes('number')
X_cat = X.select_dtypes(['object', 'category', 'string'])

In [ ]:
from sklearn.preprocessing import MinMaxScaler
SN = MinMaxScaler()
X_num_SN = pd.DataFrame(SN.fit_transform(X_num), columns=X_num.columns, index=index)

In [ ]:
X_cat_dummies = pd.get_dummies(X_cat)

: 

In [ ]:
X_encoded = pd.concat([X_num_SN, X_cat_dummies], axis=1)